In [ ]:
# Task 3: Forecast Future Market Trends for Tesla Stock

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAXResults
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import joblib

# Load Data Online
data = yf.download('TSLA', start='2015-01-01', end='2024-10-31')
data = data['Close']  # Focus on closing prices for forecasting

# Forecasting Horizon (6-12 months)
forecast_horizon = 252  # Around 12 months of trading days

# Function to plot forecast with confidence intervals
def plot_forecast(data, predictions, confidence_intervals=None, title="Forecast"):
    plt.figure(figsize=(14, 7))
    plt.plot(data, label='Historical Data')
    plt.plot(predictions.index, predictions, label='Forecast', color='orange')
    if confidence_intervals is not None:
        plt.fill_between(predictions.index, 
                         confidence_intervals.iloc[:, 0], 
                         confidence_intervals.iloc[:, 1], color='pink', alpha=0.3)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# Select model type: 'ARIMA', 'SARIMA', or 'LSTM'
model_type = 'ARIMA'  # Replace as needed

# 1. Load and Forecast with ARIMA Model
if model_type == 'ARIMA':
    arima_model = joblib.load("../models/arima_model.pkl")
    arima_forecast = arima_model.get_forecast(steps=forecast_horizon)
    arima_predictions = arima_forecast.predicted_mean
    confidence_intervals = arima_forecast.conf_int()
    plot_forecast(data, arima_predictions, confidence_intervals, "Tesla Stock Price Forecast - ARIMA")

# 2. Load and Forecast with SARIMA Model
elif model_type == 'SARIMA':
    sarima_model = joblib.load("../models/sarima_model.pkl")
    sarima_forecast = sarima_model.get_forecast(steps=forecast_horizon)
    sarima_predictions = sarima_forecast.predicted_mean
    confidence_intervals = sarima_forecast.conf_int()
    plot_forecast(data, sarima_predictions, confidence_intervals, "Tesla Stock Price Forecast - SARIMA")

# 3. Load and Forecast with LSTM Model
elif model_type == 'LSTM':
    lstm_model = load_model("../models/lstm_model.h5")
    
    # Prepare the input sequence for LSTM based on the last 60 data points
    last_data = data[-60:].values.reshape(-1, 1)
    input_sequence = TimeseriesGenerator(last_data, last_data, length=60, batch_size=1)

    # Forecast using LSTM
    lstm_forecast = []
    current_batch = last_data[-60:]
    for i in range(forecast_horizon):
        current_pred = lstm_model.predict(current_batch.reshape(1, 60, 1))[0]
        lstm_forecast.append(current_pred)
        current_batch = np.append(current_batch[1:], current_pred)

    # Convert LSTM predictions to a DataFrame with a Date index
    lstm_forecast_dates = pd.date_range(start=data.index[-1], periods=forecast_horizon + 1, freq='B')[1:]
    lstm_forecast_df = pd.DataFrame(lstm_forecast, index=lstm_forecast_dates, columns=['LSTM Forecast'])
    
    plot_forecast(data, lstm_forecast_df['LSTM Forecast'], title="Tesla Stock Price Forecast - LSTM")

# Interpretation of Forecast Results
print("Interpretation of Forecast Results:")
print("1. **Trend Analysis**: The forecast shows whether Tesla's stock is expected to move upward, downward, or remain stable.")
print("2. **Volatility and Risk**: Confidence intervals indicate potential uncertainty, with wider intervals suggesting higher volatility.")
print("3. **Market Opportunities and Risks**: Based on the forecast, identify opportunities in expected price increases and risks during high volatility periods.")
